In [14]:
import torch
import torch.nn as nn
import torchvision as vision
from torchvision.transforms import Lambda,Compose,ToTensor,Normalize,Grayscale
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

In [42]:
transform=Compose([Grayscale(),ToTensor()])
#transform=Compose([Grayscale(),ToTensor(),Lambda(lambda x:x/255.0)])
#transform=Compose([Grayscale(),ToTensor()])
dataset_train=datasets.ImageFolder("/kaggle/input/fer2013/train",transform=transform)

In [43]:
dataset_train.classes

['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

In [44]:
len(dataset_train)

28709

In [45]:
itr=iter(dataset_train)
img,label=next(itr)
img.size()

torch.Size([1, 48, 48])

In [49]:
train_loader=DataLoader(dataset_train,batch_size=64,shuffle=True)

In [51]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.relu=nn.ReLU()
        self.flatten=nn.Flatten()
        self.conv1=nn.Conv2d(in_channels=1,out_channels=32,kernel_size=3)
        self.conv2=nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3)
        self.pool1=nn.MaxPool2d(kernel_size=2)
        self.drop1=nn.Dropout()
        self.conv3=nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3)
        self.conv4=nn.Conv2d(in_channels=128,out_channels=256,kernel_size=3)
        self.pool2=nn.MaxPool2d(kernel_size=2)
        self.drop2=nn.Dropout()
        self.fc1=nn.Linear(in_features=9*9*256,out_features=1024)
        self.drop3=nn.Dropout()
        self.fc2=nn.Linear(in_features=1024,out_features=7)
    def forward(self,x):
        #input (1,48,48)
        x=self.conv1(x)
        x=self.relu(x)
        #input (32,46,46)
        x=self.conv2(x)
        x=self.relu(x)
        #input (64,44,44)
        x=self.pool1(x)
        x=self.drop1(x)
        
        #input (64,22,22)
        x=self.conv3(x)
        x=self.relu(x)
        #input (128,20,20)
        x=self.conv4(x)
        x=self.relu(x)
        #input (256,18,18)
        x=self.pool2(x)
        x=self.drop2(x)
        #input (256,9,9)
        x=self.flatten(x)
        x=self.fc1(x)
        x=self.relu(x)
        x=self.drop3(x)
        x=self.fc2(x)
        return x

In [73]:
def init_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m,nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.0)
model=Net()
model=model.cuda()
model.apply(init_weights)

Net(
  (relu): ReLU()
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (drop1): Dropout(p=0.5, inplace=False)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (drop2): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=20736, out_features=1024, bias=True)
  (drop3): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=1024, out_features=7, bias=True)
)

In [74]:
%%time
import torch.optim as optim
loss_fn=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters())
from tqdm import tqdm
epochs=10

for epoch in range(epochs):
    print(f"epoch {epoch}:",end=' ')
    train_loader=tqdm(train_loader)
   
    for imgs,labels in train_loader:
        imgs=imgs.cuda()
        labels=labels.cuda()
        outputs=model(imgs)
        optimizer.zero_grad()
        loss=loss_fn(outputs,labels)
        loss.backward()
        optimizer.step()
    print(loss.item())
        

epoch 0: 

100%|██████████| 449/449 [00:34<00:00, 12.97it/s]


1.8939415216445923
epoch 1: 

100%|██████████| 449/449 [00:33<00:00, 13.23it/s]


1.5942695140838623
epoch 2: 

...

1.2053191661834717
epoch 8: 

100%|██████████| 449/449 [00:33<00:00, 13.29it/s]


1.1751790046691895
epoch 9: 

100%|██████████| 449/449 [00:33<00:00, 13.57it/s]


1.2949873208999634
CPU times: user 2min 30s, sys: 20.5 s, total: 2min 51s
Wall time: 5min 47s


In [58]:
dataset_test=datasets.ImageFolder("/kaggle/input/fer2013/test",transform=transform)
loader_test=DataLoader(dataset_test,batch_size=128,shuffle=False)

In [61]:
total=len(dataset_test)
correct=0
for imgs,labels in loader_test:
    imgs=imgs.cuda()
    labels=labels.cuda()
    outputs=model(imgs)
    _,predicted=torch.max(outputs.data,1)
    tmp=(predicted==labels).sum()
    #print(tmp/128)
    correct+=tmp


tensor(0.4141, device='cuda:0')
tensor(0.3594, device='cuda:0')
tensor(0.3906, device='cuda:0')
tensor(0.4453, device='cuda:0')
tensor(0.4219, device='cuda:0')
tensor(0.3750, device='cuda:0')
tensor(0.4922, device='cuda:0')
tensor(0.2969, device='cuda:0')
tensor(0.2812, device='cuda:0')
tensor(0.3828, device='cuda:0')
tensor(0.3359, device='cuda:0')
tensor(0.2734, device='cuda:0')
tensor(0.2812, device='cuda:0')
tensor(0.2500, device='cuda:0')
tensor(0.3125, device='cuda:0')
tensor(0.3359, device='cuda:0')
tensor(0.6484, device='cuda:0')
tensor(0.7656, device='cuda:0')
tensor(0.8281, device='cuda:0')
tensor(0.7969, device='cuda:0')
tensor(0.8047, device='cuda:0')
tensor(0.7500, device='cuda:0')
tensor(0.8438, device='cuda:0')
tensor(0.8359, device='cuda:0')
tensor(0.7734, device='cuda:0')
tensor(0.7734, device='cuda:0')
tensor(0.7266, device='cuda:0')
tensor(0.7891, device='cuda:0')
tensor(0.7734, device='cuda:0')
tensor(0.7500, device='cuda:0')
tensor(0.6406, device='cuda:0')
tensor(0

In [56]:
print(correct/total)

tensor(0.5302, device='cuda:0')


In [57]:
print(total)

7178
